In [1]:
import sys
import os

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import numpy as np
import os
import io
import cv2

import torch

from config import (Config,
                    eval_dict_leaf)

from demo_utils import (retrieve_text,
                  _frame_from_video,
                  setup_internvideo2)

/users/piyush/install/miniconda3/envs/vl/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-06-06 18:17:20,400] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [4]:
video = cv2.VideoCapture('example1.mp4')
frames = [x for x in _frame_from_video(video)]

In [5]:
text_candidates = ["A playful dog and its owner wrestle in the snowy yard, chasing each other with joyous abandon.",
                   "A man in a gray coat walks through the snowy landscape, pulling a sleigh loaded with toys.",
                   "A person dressed in a blue jacket shovels the snow-covered pavement outside their house.",
                   "A pet dog excitedly runs through the snowy yard, chasing a toy thrown by its owner.",
                   "A person stands on the snowy floor, pushing a sled loaded with blankets, preparing for a fun-filled ride.",
                   "A man in a gray hat and coat walks through the snowy yard, carefully navigating around the trees.",
                   "A playful dog slides down a snowy hill, wagging its tail with delight.",
                   "A person in a blue jacket walks their pet on a leash, enjoying a peaceful winter walk among the trees.",
                   "A man in a gray sweater plays fetch with his dog in the snowy yard, throwing a toy and watching it run.",
                   "A person bundled up in a blanket walks through the snowy landscape, enjoying the serene winter scenery."]

In [6]:
config = Config.from_file('internvideo2_stage2_config.py')
config = eval_dict_leaf(config)

In [8]:
model_pth = '/work/piyush/pretrained_checkpoints/LargeModels/InternVideo/1B_clip.pth'
config['pretrained_path'] = model_pth

In [7]:
model_pth = "/work/piyush/pretrained_checkpoints/LargeModels/InternVideo/InternVideo2-stage2_1b-224p-f4.pt"
config["model"]["vision_encoder"]["pretrained"] = model_pth

In [8]:
config['TextEncoders']['bert_large']['config'] = "../configs/config_bert_large.json"

In [9]:
intern_model, tokenizer = setup_internvideo2(config)

/users/piyush/install/miniconda3/envs/vl/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
texts, probs = retrieve_text(frames, text_candidates, model=intern_model, topk=5, config=config)

for t, p in zip(texts, probs):
    print(f'text: {t} ~ prob: {p:.4f}')

/users/piyush/install/miniconda3/envs/vl/lib/python3.9/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


text: A man in a gray hat and coat walks through the snowy yard, carefully navigating around the trees. ~ prob: 0.6464
text: A person in a blue jacket walks their pet on a leash, enjoying a peaceful winter walk among the trees. ~ prob: 0.1907
text: A man in a gray sweater plays fetch with his dog in the snowy yard, throwing a toy and watching it run. ~ prob: 0.0718
text: A person bundled up in a blanket walks through the snowy landscape, enjoying the serene winter scenery. ~ prob: 0.0431
text: A man in a gray coat walks through the snowy landscape, pulling a sleigh loaded with toys. ~ prob: 0.0317


In [11]:
video_path = "/users/piyush/projects/TimeBound.v1/sample_data/folding_paper.mp4"
video = cv2.VideoCapture(video_path)
frames = [x for x in _frame_from_video(video)]

In [12]:
text_candidates = [
    "Someone is folding a paper.",
    "Someone is unfolding a paper.",
]

In [14]:
texts, probs = retrieve_text(frames, text_candidates, model=intern_model, topk=2, config=config)

for t, p in zip(texts, probs):
    print(f'text: {t} ~ prob: {p:.4f}')

text: Someone is unfolding a paper. ~ prob: 0.7811
text: Someone is folding a paper. ~ prob: 0.2189
